<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
(x_full, y_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(x_full,y_full,test_size=0.2,random_state=13)

In [ ]:
X_train.shape

In [ ]:
from functools import partial
Conv2D=partial(
    tf.keras.layers.Conv2D,
    kernel_size=3,
    filters=64,
    padding='same',
    strides=1,
    activation='relu',
    kernel_initializer='he_normal'
)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28,1)),
    Conv2D(),
    tf.keras.layers.MaxPool2D(),
    Conv2D(filters=128),
    Conv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    Conv2D(filters=256),
    Conv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(64, kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(10, activation='softmax')


])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid))

In [ ]:
import numpy as np
X_train = np.expand_dims(X_train, axis = -1)
X_valid = np.expand_dims(X_valid, axis = -1)

In [ ]:
model1=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28, 1)),
    Conv2D(),
    tf.keras.layers.MaxPool2D(),
    Conv2D(filters=128),
    Conv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    Conv2D(filters=256),
    Conv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64,kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
])



In [ ]:
tf.keras.backend.clear_session()
class MonteCarlo(tf.keras.layers.Dropout):
  def call(self,inputs):
    return super().call(inputs,training=True)
Dropout=tf.keras.layers.Dropout
mc_model=tf.keras.Sequential([
    MonteCarlo(layer.rate) if isinstance(layer,Dropout) else layer for layer in model1.layers
])

mc_model.set_weights(model1.get_weights())


mc_model.summary()

In [ ]:
mc_model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3,momentum=0.9,nesterov=True),
                 metrics=['accuracy'])

In [ ]:
mc_model.fit(X_train,y_train, epochs=7, validation_data=(X_valid,y_valid))

In [ ]:
from functools import partial
defaulConv2D=partial(tf.keras.layers.Conv2d,kernel_size=3, strides=1,padding='same',
                     kernel_initializer='he_normal', use_bias=False)

class Residual(tf.keras.layers.Layer):
  def __init__(self,filters,strides,**kwargs):
    super().__init__(**kwargs)
    self.activation=tf.keras.activations.get(activation)
    self.main_layer=[
        defaulConv2D(filters,strides=strides),
        tf.keras.layers.BatchNormalization(),
        self.activation,
        defaulConv2D(filters),
        tf.keras.layers.BatchNormalization()
    ]

    self.skip_layer=[]
    if strides>1:
      self.skip_layer=[
          defaulConv2D(filters,kernel_size=1,strides=strides),
          tf.keras.layers.BatchNormalization()
      ]


    def call(self,inputs):
      Z=inputs
      for layer in self.main_layer:
        Z=layer(Z)
      skip_Z=inputs
      for layer in self.skip_layer:
        skip_Z=layer(skip_Z)
      return self.activation(Z+skip_Z)



In [ ]:
class Monte_Carlo(tf.keras.layers.Dropout):
  def__init__(self,inputs,training=True):
    return super().call(inputs, training=True)
Dropout=tf.keras.layers.Dropout
mc_model=tf.keras.Sequential([
    Monte_Carlo(layer.rate) if isinstance(layer,Dropout) else layer for layer in model.layers
])

mc_model.set_weights(model.get_weights())